In [ ]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

import getdist.plots as gdplt
from cobaya.model import get_model
from cobaya import run

# Section 1: Simple example

In this first section, we learn the basics of Cobaya. Let's say that we have data points and want to perform a linear regression such that our model is
$y=\alpha x + \beta$. We will firstly generate random data and use Cobaya to fit $\alpha$ and $\beta$. On top of that, let's pretend that the value (called $\gamma$) of $y(x=1)$ is of physical interest and let's make it a derived parameter.

In [ ]:
# Define the model

def my_model(x, alpha, beta):
    return ...

In [ ]:
# Choose values for alpha, beta and add some Gaussian noise with amplitude epsilon

alpha = 1.
beta = 0.5
epsilon = 0.05

# Create data

x_data = 
y_data = 

# Compute covariance (assuming data to be uncorrelated)

cov = np.diag(epsilon**2*np.ones(len(x_data)))
inv_cov = np.linalg.inv(cov)

In [ ]:
# This is how our synthetic data look like as well as the theoretical, noiseless, underlying model

plt.figure()
plt.errorbar(x_data,y_data,np.sqrt(np.diag(cov)), capsize=3, label="Data")
plt.plot(x_data,my_model(x_data, alpha, beta), label="Theoretical model")
plt.grid()
plt.legend()

In [ ]:
# Create a function that returns the Gaussian log-likelihood

def my_log_likelihood(alpha, beta):
    
    y_theoretical = my_model(x_data, alpha, beta)
    log_like = -0.5*(y_data-y_theoretical)@inv_cov@(y_data-y_theoretical).T
    
    return log_like

# Create a function that returns the derived parameters

def my_gamma(alpha, beta):
    gamma = my_model(1., alpha, beta)
    return gamma
    

In [ ]:
# Create the cobaya info dictionnary

info = {}

info["likelihood"] = {"experiment": my_log_likelihood}

info["params"] = {
    "alpha": {
        "prior": {"min": ..., "max": ...},
        "ref": ...,
        "proposal": ...,
        "latex": r"\alpha"
    },
    "beta": {
        "prior": {"min": ..., "max": ...},
        "ref": ...,
        "proposal": ...,
        "latex": r"\beta"
     },
    "gamma": {
        "derived": my_gamma,
        "latex": r"\gamma"
    }
}

info["sampler"] = {
    "mcmc": {
        "Rminus1_stop": 0.01,
        "max_tries": 1000
    }
}


In [ ]:
# Test our cobaya model before running the mcmc

model = get_model(info)

# Choose a random point to perform tests

random_point = model.prior.sample(ignore_external=True)[0]

print("Our random point is:", random_point)

# Look at the log-prior, log-likelihoods, derived parameters and log-posterior
print("The log-prior is:", model.logprior(random_point))
print("The log-likelihoods and derived parameters are:",
      model.loglikes(random_point))

print("The log-posterior is:", model.logpost(random_point))

In [ ]:
# Run the mcmc!

updated_info, sampler = run(info)

In [ ]:
# Extract the samples, provide right format for getdist and remove burn-in

gd_sample = sampler.products(to_getdist=True, skip_samples=0.3)["sample"]

In [ ]:
# Make triangle plot

gamma_theoretical = my_gamma(alpha, beta)

gdplot = gdplt.get_subplot_plotter()
gdplot.triangle_plot(gd_sample, ["alpha", "beta", "gamma"], filled=True, 
                     markers={'alpha': alpha, 'beta': beta, 'gamma': gamma_theoretical})

Even if up to now and in the following sections we will always define the info dictionary through python, it is in practice more common to use yaml files and likelihoods defined in their own python files. I provided all the required files for the simple example we've just done. In more detail:

- The model and likelihood functions we wrote above are replaced by the **simple_likelihood.py** file. Take some time to have a look at it.
- The data and covariance matrix have already been computed and are loaded by the likelihood file.
- The info dictionary is replaced by the simple_likelihood.yaml file. Beware some values are missing!

Can you run the MCMC and find the values of $\alpha$ and $\beta$ I used to generate the data file?

In [ ]:
# run the MCMC using the yaml file and find the values of alpha and beta


# Section 2: Oversampling, dragging

In many cases, cosmological likelihoods can be split into several pieces that can have very different execution times. Some parameters may only contribute to the fastest likelihoods and Cobaya has two different interesting techniques that allow to take advantage from those "fast parameters" to make the chains converge faster. Those two techniques are called "oversampling" and "dragging". 

In this section, we will see how to use those techniques with Cobaya. We first consider an example based on the fibonacci sequence where the likelihood takes a few seconds to run and then use the oversampling and dragging methods to speed up the convergence of the chain.

Let's assume that we have two independent observations $x$ and $y$. We know that we can theoretically modeled the two observations as follow:

$x = \alpha*\beta$ where $\alpha$ and $\beta$ are the two parameters we want to estimate.

$y=\Phi_n(\alpha)$ where $\Phi_n$ is the $n^\rm{th}$ element of the Fibonacci sequence and depends on the initial condition which for theoretical reasons is $\alpha$.

In [ ]:
# We define the fiducial values used to generate the data

alpha = 1
beta = 2
n = 27

def fibo(n, alpha):
    if n < 2:
        return alpha
    else:
        return fibo(n-1, alpha) + fibo(n-2, alpha)

# We generate the data
x = alpha*beta
y = fibo(n, alpha)
y_data = np.array([x, y])

# We have this covariance matrix for those observations
cov = np.diag([1e-4, 1e7])
inv_cov = np.linalg.inv(cov)

In [ ]:
# We then define the log-likelihood:

def my_log_likelihood(alpha, beta):
    

In [ ]:
# We define the info dictionary

info = {}

info["likelihood"] = {"experiment": my_log_likelihood}

info["params"] = {
    "alpha": {
        "prior": {"min": 0.9, "max": 1.1},
        "ref": 1.,
        "proposal": 0.0001,
        "latex": r"\alpha"
    },
    "beta": {
        "prior": {"min": 0, "max": 10},
        "ref": 2.,
        "proposal": 0.01,
        "latex": r"\beta"
     }
}

info["sampler"] = {
    "mcmc": {
        "Rminus1_stop": 0.1,
        "max_tries": 1000
    }
}

In [ ]:
# Run the mcmc and time it
import time

t1 = time.time()
updated_info, sampler = run(info)
t2 = time.time()
print(t2-t1)

We will now use the oversampling and dragging methods to speed up the convergence. 

**Task:** 
Create two different likelihoods for the two observations $x$ and $y$ and re-run the MCMC. By default, Cobaya will automatically detect if one likelihood is faster than the other and will apply the oversampling method if that is relevant. Then, to use the dragging method instead, set the 'drag' flag to True in the 'mcmc' dictionary.

# Section 3: Interaction with CAMB

In Cobaya, you can create theory classes which can then be used by the likelihoods you create to compute relevant quantities to model the observations you want to fit. In the case of the CMB for instance, CAMB is used to model the power spectra which can then be compared to the observations inside your likelihoods. In Cobaya, a CAMB wrapper has already been implemented so that you can directly specify 'camb' in the 'theory' field of the info dictionary. In this section, we will see in more details how it goes.

In [ ]:
# We define cosmological parameters here

H0 = 67.5
ombh2 = 0.02242
omch2 = 0.11933
tau = 0.0561
As = 2.105e-9
ns = 0.9665

**Task :**
Use camb to generate some synthetic data (for instance, only TT power spectra with lmax=2000). In order to gain some execution time, you can specify:
- AccuracyBoost=0.6
- NonLinear='NonLinear_none'
when setting parameters. Those degraded options will speed up the code.

In [ ]:
# Compute a covariance matrix. Let's assume a full-sky cosmic-variance limited survey

ls = np.arange(powers['total'].shape[0])[2:lmax]
cl_tt = powers['total'][2:lmax,0]
cov = np.diag(2/(2*(ls+1))*cl_tt**2)
inv_cov = np.linalg.inv(cov)

In [ ]:
# Plot
sigma = np.diag(np.sqrt(cov))
plt.figure()
plt.fill_between(ls, cl_tt-sigma, cl_tt+sigma, color='C0', alpha=1)
plt.fill_between(ls, cl_tt-2*sigma, cl_tt+2*sigma, color='C0', alpha=0.5)
plt.xlabel(r"$\ell$", fontsize=14)
plt.ylabel(r"$C_\ell^{TT}$", fontsize=14)

plt.figure()
plt.fill_between(ls, cl_tt-sigma, cl_tt+sigma, color='C0', alpha=1)
plt.fill_between(ls, cl_tt-2*sigma, cl_tt+2*sigma, color='C0', alpha=0.5)
plt.semilogx()
plt.xlabel(r"$\ell$", fontsize=14)
plt.ylabel(r"$C_\ell^{TT}$", fontsize=14)

In [ ]:
# We create a Likelihood class. Note that the logp method must always be defined 
# while initialize and get_requirements are optional

from cobaya.likelihood import Likelihood

class my_likelihood_class(Likelihood):

    def initialize(self):
        self.data = ...
        self.inv_covmat = ...
        self.lmin = 2
        self.lmax = lmax
        
    # The get_requirements method is used to specify what the likelihood needs the theory class to provide
    # Here we need CAMB to provide the TT power spectra up to l=lmax
    def get_requirements(self):
        return {'Cl': {'tt': lmax}}

    # Here we compute the log-likelihood
    def logp(self, **params_values):
        cls = self.provider.get_Cl(ell_factor=True, units="1") 
        # This is known because we requested it in get_requirements
        
        ...
        return ...

In [ ]:
# Let's say we only want to sample w, the equation of state parameter of dark energy

info = {}

info['theory'] = {'camb': 
                     {'extra_args': {
                         'ombh2': ombh2,
                         'omch2': omch2,
                         'tau': tau,
                         'As': As,
                         'ns': ns,
                         'H0': H0,
                         'AccuracyBoost': 0.6,
                         'NonLinear': 'NonLinear_none',
                         'lmax': lmax,
                         'dark_energy_model': 'DarkEnergyPPF'
                         }
                     }
                 }

info["likelihood"] = ...

info["params"] = {
    "w": {
        "prior": {"min": -3, "max": 3},
        "ref": -1,
        "proposal": 0.001,
        "latex": r"w"
    }
}

info["sampler"] = {
    "mcmc": {
        # We modify the convergence criteria so that the run is not too long
        "Rminus1_stop": 0.25,
        "Rminus1_cl_stop": 0.5,
        "max_tries": 1000
    }
}

In [ ]:
# For your information (also useful for following sections), this is what CAMB can provide

model = get_model(info)

# Those are the parameters that CAMB can provide (in a likelihood class for instance)
print(model.theory['camb'].get_can_provide_params())

# Those are the functions of CAMB that can be called (in a likelihood class for instance)
print(model.theory['camb'].get_can_provide_methods())

In [ ]:
updated_info, sampler = run(info)

In [ ]:
gd_sample = sampler.products(to_getdist=True, skip_samples=0.3)["sample"]

gdplot = gdplt.get_subplot_plotter(width_inch=5)
gdplot.plot_1d(gd_sample, "w", title_limit=1)
gdplot.add_x_marker(-1)

# Section 4 : Use cobaya's included likelihoods. Example: Planck

Cobaya has a number of already implemented likelihoods. For instance, you can directly used the latest Planck likelihood without any effort. In particular, the calibration and nuisance parameters will be automatically added, together with the priors and proposal values!

In [ ]:
# Create the cobaya info dictionnary

info = {}

info['theory'] = {'camb': 
                     {'extra_args': {
                         'ombh2': ombh2,
                         'omch2': omch2,
                         'tau': tau,
                         'As': As,
                         'ns': ns,
                         'H0': H0,
                         'AccuracyBoost': 0.6,
                         'NonLinear': 'NonLinear_none',
                         'lmax': lmax,
                         'dark_energy_model': 'DarkEnergyPPF'
                         }
                     }
                 }

info["likelihood"] = {'planck_NPIPE_highl_CamSpec.TTTEEE': None}

info["params"] = {
    "w": {
        "prior": {"min": -3, "max": 0},
        "ref": -1.,
        "proposal": 0.001,
        "latex": r"w_0"
    }
}

covmat = np.load("covmat_w_Planck.npy")

info["sampler"] = {
    "mcmc": {
        "Rminus1_stop": 0.3,
        "Rminus1_cl_stop": 0.5,
        "max_tries": 1000,
        "learn_proposal_Rminus1_max": 0.4,
        "covmat": covmat,
        "covmat_params": ['w','A_planck', 'amp_143', 'amp_217', 'amp_143x217', 'n_143', 'n_217', 'n_143x217', 'calTE', 'calEE']
    }
}

In [ ]:
# Pay attention to the fact that Cobaya is automatically using oversampling!

updated_info, sampler = run(info)

In [ ]:
gd_sample = sampler.products(to_getdist=True, skip_samples=0.3)["sample"]

gdplot = gdplt.get_subplot_plotter()
gdplot.triangle_plot(gd_sample, 
    ['w','A_planck', 'amp_143', 'amp_217', 'amp_143x217', 'n_143', 'n_217', 'n_143x217', 'calTE', 'calEE'], 
    filled=True)

# Section 5: Write your own (more advanced) likelihood class

In this more advanced section, we will use what we have used so far to put some constraints on four parameters, namely: 
- the Hubble constant $H_0$
- the dark matter density $\omega_c=\Omega_ch^2$
- the dark energy equation of state CPL parameters $w_0$ and $w_a$ where $w(a) = w_0 + (1-a)w_a$

We will use several observations to constrain those parameters and hence perform a multi probe analysis. The probe we will consider are:
- a (very) approximate CMB likelihood
- a (very) approximate BAO likelihood
- a local measurement of $H_0$

**Probe 1: CMB**

We could use the Planck likelihood as it is already implemented in Cobaya, however computing the power spectra through CAMB takes too much time for the purpose of this section. We assume that the CMB likelihood consists in two quantities
- the angular scale $\theta_\star=1.04109 \pm 0.0003$ which is related to the position of the first peak in the CMB power spectrum
- the redshift of recombination $z_\star = 1089.95 \pm 0.27$

**Task :**
Implement this likelihood and run the MCMC for the four parameters mentioned above. Show the triangle plot.

In [ ]:
class CMB(Likelihood):

    def initialize(self):
        

    def get_requirements(self):
        

    def logp(self, **params_values):
        

info = {}

info['theory'] = {'camb': 
                     {'extra_args': {
                         ...
                         'dark_energy_model': 'DarkEnergyPPF'
                         }
                     }
                 }

info["likelihood"] = ...

info["params"] = ...

info["sampler"] = {
    "mcmc": {
        "Rminus1_stop": 0.1,
        "max_tries": 1000,
    }
}

**Probe 2: BAO**

We use data from "DESI 2024 VI: Cosmological Constraints from the Measurements of Baryon Acoustic Oscillations". We assume all the measurements to be independent. 

Here are the measurements we will use:
- $D_V/r_d(z=0.295) = 7.93 \pm 0.15$
- $D_V/r_d(z=1.491) = 26.07 \pm 0.67$

- $D_M/r_d(z=0.510) = 13.62 \pm 0.25$
- $D_M/r_d(z=0.706) = 16.85 \pm 0.32$
- $D_M/r_d(z=0.930) = 21.71 \pm 0.28$
- $D_M/r_d(z=1.317) = 27.79 \pm 0.69$
- $D_M/r_d(z=2.330) = 39.71 \pm 0.94$

- $D_H/r_d(z=0.510) = 20.98 \pm 0.61$
- $D_H/r_d(z=0.706) = 20.08 \pm 0.60$
- $D_H/r_d(z=0.930) = 17.88 \pm 0.35$
- $D_H/r_d(z=1.317) = 13.82 \pm 0.42$
- $D_H/r_d(z=2.330) = 8.52 \pm 0.17$

**Task:** 

Implement this likelihood, run the MCMC and make the triangle plot.

**Probe 3: Local measurement of $H_0$**

We finally add a last measurement which is based on the local measurement of $H_0$ with the distance-ladder method. The latest measurement (https://arxiv.org/abs/2112.04510) is 

$H_0 = 73.04 \pm 1.04$ and can be added as a Gaussian likelihood.

**Task:**

Implement this likelihood and run the MCMC with the combination of the three likelihoods (CMB + BAO + local H0), make the triangle plot.

# Bonus section: try using Cobaya for a simple model relevant to your research area